#**Importing Necessary Libraries**

In [ ]:
import numpy as np
import pandas as pd

#**Loading the Combined Cycle Power Plant Dataset**

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of **hourly average ambient variables Temperature (AT)**, **Ambient Pressure (AP)**, **Relative Humidity (RH)** and **Exhaust Vacuum (V)** to predict the net **hourly electrical energy output (PE)** of the plant.

* **For more information visit below link:**
Link: https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant





In [ ]:
Powerplant_data = pd.read_excel('/content/Folds5x2_pp.xlsx')
Powerplant_data.head(5)

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


#**Accessing the Column Names in the Dataset**

In [ ]:
Powerplant_data.columns

Index(['AT', 'V', 'AP', 'RH', 'PE'], dtype='object')

#**Finding the Shape of the Dataset**

In [ ]:
Powerplant_data.shape

(9568, 5)

In [ ]:
Powerplant_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


#**Checking Missing Values**

In [ ]:
Powerplant_data.isna().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

# **Number of Unique Items in Each Column**

In [ ]:
Powerplant_data.nunique()

AT    2773
V      634
AP    2517
RH    4546
PE    4836
dtype: int64

#**Seperating Label from Data**

In [ ]:
X = Powerplant_data.iloc[:, :-1].values
y = Powerplant_data.iloc[:, -1].values

or

In [ ]:
# y = Powerplant_data['PE']
# X = Powerplant_data.drop(['PE'],axis=1)

#**Splitting the Data into Training and Testing**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.2, shuffle=True)

In [ ]:
print("Shape of the X_train", X_train.shape)
print("Shape of the X_test", X_test.shape)
print("Shape of the X_val", X_val.shape)
print("Shape of the y_train", y_train.shape)
print("Shape of the y_test", y_test.shape)
print("Shape of the y_val", y_val.shape)

Shape of the X_train (6123, 4)
Shape of the X_test (1914, 4)
Shape of the X_val (1531, 4)
Shape of the y_train (6123,)
Shape of the y_test (1914,)
Shape of the y_val (1531,)


#**Building the ANN Model**

In [ ]:
# sequential model to initialise our ann and dense module to build the layers
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform'))

# **Compiling and Fitting the Model**

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['MeanSquaredLogarithmicError'])

# Fitting the ANN to the Training set
model = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200,validation_data=(X_val, y_val),
              shuffle=True)

Epoch 1/200
192/192 [==============================] - 1s 3ms/step - loss: 27.4878 - mean_squared_logarithmic_error: 1.3154e-04 - val_loss: 24.9590 - val_mean_squared_logarithmic_error: 1.1958e-04
Epoch 2/200
192/192 [==============================] - 0s 2ms/step - loss: 28.0632 - mean_squared_logarithmic_error: 1.3424e-04 - val_loss: 32.0421 - val_mean_squared_logarithmic_error: 1.5361e-04
Epoch 3/200
192/192 [==============================] - 0s 2ms/step - loss: 27.6081 - mean_squared_logarithmic_error: 1.3208e-04 - val_loss: 26.8964 - val_mean_squared_logarithmic_error: 1.2891e-04
Epoch 4/200
192/192 [==============================] - 0s 2ms/step - loss: 27.7473 - mean_squared_logarithmic_error: 1.3284e-04 - val_loss: 25.3694 - val_mean_squared_logarithmic_error: 1.2157e-04
Epoch 5/200
192/192 [==============================] - 0s 2ms/step - loss: 28.1495 - mean_squared_logarithmic_error: 1.3468e-04 - val_loss: 24.7678 - val_mean_squared_logarithmic_error: 1.1869e-04
Epoch 6/200
192

#**Testing the Model**

In [ ]:
y_pred = classifier.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

60/60 [==============================] - 0s 1ms/step
[[469.98 475.36]
 [458.92 454.86]
 [438.71 435.36]
 ...
 [438.08 438.09]
 [451.59 447.96]
 [472.39 471.63]]


#**Metric values**

**MAE (Mean Absolute Error) :-**

$$MAE = (\frac{1}{n})\sum_{i=1}^{n}\left | y_{i} - \hat y_{i} \right |$$

**MSE (Mean Square Error) :-**

$$MSE = (\frac{1}{n})\sum_{i=1}^{n}\left ( y_{i} - \hat y_{i} \right )^2$$

where y = actual value in the data set ;  $\hat y$ = value computed by solving the regression equation

**RMSE (Root Mean Square Error) :-**

$$RMSE = \sqrt{(\frac{1}{n})\sum_{i=1}^{n}\left ( y_{i} - \hat y_{i} \right )^2}$$


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sklearn.metrics
from math import sqrt
mae_no = sklearn.metrics.mean_absolute_error(y_test,classifier.predict(X_test))
mse_no = sklearn.metrics.mean_squared_error(y_test,classifier.predict(X_test))
rms = sqrt(sklearn.metrics.mean_squared_error(y_test,classifier.predict(X_test)))

60/60 [==============================] - 0s 2ms/step


In [ ]:
print('Mean Absolute Error     :',mae_no)
print('Mean Square Error       :',mse_no)
print('Root Mean Square Error:', rms)

Mean Absolute Error     : 4.0557620548579125
Mean Square Error       : 25.97531418610697
Root Mean Square Error: 5.096598295540563


# **Self Work**

In [ ]:
#compiling model with changes

In [ ]:
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'leaky_relu', input_dim = 4))

# Adding the second hidden layer
classifier.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the third hidden layer
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Adding the forth hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'leaky_relu'))


# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['MeanSquaredLogarithmicError'])

# Fitting the ANN to the Training set
model = classifier.fit(X_train, y_train, batch_size = 32, epochs = 30,validation_data=(X_val, y_val),
              shuffle=True)

Epoch 1/30
192/192 [==============================] - 2s 3ms/step - loss: 147729.8906 - mean_squared_logarithmic_error: 9.3734 - val_loss: 49516.7695 - val_mean_squared_logarithmic_error: 0.4465
Epoch 2/30
192/192 [==============================] - 1s 4ms/step - loss: 10852.0938 - mean_squared_logarithmic_error: 0.0799 - val_loss: 326.9146 - val_mean_squared_logarithmic_error: 0.0015
Epoch 3/30
192/192 [==============================] - 1s 6ms/step - loss: 293.2028 - mean_squared_logarithmic_error: 0.0014 - val_loss: 291.7021 - val_mean_squared_logarithmic_error: 0.0014
Epoch 4/30
192/192 [==============================] - 1s 7ms/step - loss: 290.4230 - mean_squared_logarithmic_error: 0.0014 - val_loss: 291.4344 - val_mean_squared_logarithmic_error: 0.0014
Epoch 5/30
192/192 [==============================] - 1s 3ms/step - loss: 290.2988 - mean_squared_logarithmic_error: 0.0014 - val_loss: 291.6107 - val_mean_squared_logarithmic_error: 0.0014
Epoch 6/30
192/192 [=======================